In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import os
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from tkinter import filedialog
from tkinter import simpledialog
from tkinter import messagebox
from tkinter import ttk
from mayavi import mlab

In [2]:
#Define a function to select the data folder and return the path
def select_folder():
    root = tk.Tk()
    root.withdraw()
    folder_selected = tk.filedialog.askdirectory()
    return folder_selected

In [3]:
#Define a function to read the dicom files, order them by InstanceNumber and return a numpy array and the metadata of the files
def read_dicom(folder):
    files = os.listdir(folder)
    files = [os.path.join(folder, file) for file in files]
    files.sort(key=lambda x: int(pydicom.dcmread(x).InstanceNumber))
    data = []
    for file in files:
        ds = pydicom.dcmread(os.path.join(folder, file))
        data.append(ds.pixel_array)
    ds = pydicom.dcmread(os.path.join(folder, files[0]))
    slices = np.array(data)
    return slices, ds

In [4]:
#Define a function to allow the user to select the HU for the tissue of interest
def select_HU():
    root = tk.Tk()
    root.withdraw()
    HU_selected = tk.simpledialog.askinteger("HU selection", "Select the HU for the tissue of interest")
    return HU_selected

In [5]:
#Define a function to adjust the contrast of the image based on RescaleSlope and RescaleIntercept
def adjust_contrast(slices,ds):
    intercept = ds.RescaleIntercept
    slope = ds.RescaleSlope
    ct = slices * slope + intercept
    return ct

In [6]:
#Define a function to keep only the pixels above the HU selected by the user and normalize the image
def normalize_image(ct, HU):
    ct[ct < HU] = 0
    ct = ct - np.min(ct)
    ct = ct / np.max(ct)
    return ct

In [7]:
#Plot 3 random slices of the image
def plot_slices(ct):
    plt.figure()
    plt.subplot(1,3,1)
    plt.imshow(ct[int(ct.shape[0]/3),:,:], cmap = 'gray')
    plt.subplot(1,3,2)
    plt.imshow(ct[int(ct.shape[0]/2),:,:], cmap = 'gray')
    plt.subplot(1,3,3)
    plt.imshow(ct[int(ct.shape[0]*2/3),:,:], cmap = 'gray')
    plt.show()

In [8]:
#Create a 3D model of the images using the mayavi
def plot_3D(ct):
    mlab.figure(bgcolor=(0, 0, 0), size=(800, 800))
    src = mlab.pipeline.scalar_field(ct)
    mlab.pipeline.volume(src, vmin=0, vmax=1)
    mlab.show()

In [9]:
#Define a loop to allow the user to select multiple folders and create the 3D model
def loop():
    while True:
        folder = select_folder()
        slices, ds = read_dicom(folder)
        ct = adjust_contrast(slices,ds)
        HU = select_HU()
        ct = normalize_image(ct, HU)
        plot_3D(ct)
        plot_slices(ct)
        plt.show()
        answer = input("Do you want to select another folder? (y/n)")
        if answer == 'n':
            break

In [ ]:
#Create a Tk App to allow the user to select the folder and the HU and show the results in a window
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.title("CT Viewer")
        self.geometry("800x600")
        self.folder = tk.StringVar()
        self.HU = tk.StringVar()
        self.folder_selected = tk.StringVar()
        self.HU_selected = tk.StringVar()
        self.folder_selected.set("No folder selected")
        self.HU_selected.set("No HU selected")
        self.folder_label = tk.Label(self, textvariable = self.folder_selected)
        self.folder_label.pack()
        self.HU_label = tk.Label(self, textvariable = self.HU_selected)
        self.HU_label.pack()
        self.folder_button = tk.Button(self, text = "Select folder", command = self.select_folder)
        self.folder_button.pack()
        self.HU_button = tk.Button(self, text = "Select HU", command = self.select_HU)
        self.HU_button.pack()
        self.plot_button = tk.Button(self, text = "Plot", command = self.plot)
        self.plot_button.pack()
        self.plot_3D_button = tk.Button(self, text = "Plot 3D", command = self.plot_3D)
        self.plot_3D_button.pack()
        self.quit_button = tk.Button(self, text = "Quit", command = self.quit)
        self.quit_button.pack()
    def select_folder(self):
        self.folder_selected.set(select_folder())
    def select_HU(self):
        self.HU_selected.set(select_HU())
    def plot(self):
        slices, ds = read_dicom(self.folder_selected.get())
        ct = adjust_contrast(slices,ds)
        ct = normalize_image(ct, int(self.HU_selected.get()))
        plot_slices(ct)
    def plot_3D(self):
        slices, ds = read_dicom(self.folder_selected.get())
        ct = adjust_contrast(slices,ds)
        ct = normalize_image(ct, int(self.HU_selected.get()))
        plot_3D(ct)


In [ ]:
app = App()

In [ ]:
app.mainloop()

In [15]:
app = App()

In [16]:
app.mainloop()

In [10]:
loop()

KeyboardInterrupt: 